In [ ]:
!pip install transformers
!pip install sacrebleu sentencepiece
!pip install huggingface_hub
!pip install sacremoses

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import os
import re
import glob
import pandas as pd
from natsort import natsorted

from huggingface_hub import notebook_login
from transformers import pipeline

In [2]:
notebook_login()

## Traduzione

Configuro modelli HuggingFace per la traduzione da Inglese a Italiano

In [3]:
# Modello Traduzione Inglese-Italiano
en_model_checkpoint = "Helsinki-NLP/opus-mt-it-en"
translator_en = pipeline("translation", model=en_model_checkpoint)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-it-en.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


### Configuro Classe per la traduzione

### Configuro Classe per la traduzione

In [ ]:
class Translate:
  def __init__(self,
               orig_filenamepath, 
               trad_filenamepath_en, 
               translator_en,

               path_out):
    
    self.orig_filenamepath = orig_filenamepath
    self.trad_filenamepath_en = trad_filenamepath_en

    self.translator_en = translator_en

    self.path_out = path_out

  def text_cleaning(self, text):
    text = re.sub('\([^)]*\)', '', text)

    return text

  def translate(self):
    df_trad = pd.DataFrame(columns=['Original', 'Translate_EN', 'Translate_IT_1'])

    for file_orig, file_trad_en in zip(natsorted(glob.glob(self.orig_filenamepath)), 
                                       natsorted(glob.glob(self.trad_filenamepath_en))):
      
      df_orig = pd.read_csv(file_orig, 
                            header=None, 
                            sep='\\n', 
                            names=['Original'], 
                            engine='python')
            
      df_trad_en = pd.read_csv(file_trad_en, 
                               header=None, 
                               sep='\\n', 
                               names=['Translate_from_EN'], 
                               engine='python')
      

      trad_en = self.translator_en([self.text_cleaning(text) for text in df_trad_en['Translate_from_EN']])

      df_trad['Original'] = df_orig['Original']
      df_trad['Translate_EN'] = df_trad_en['Translate_from_EN']
      df_trad['Translate_IT_1'] = pd.DataFrame([text['translation_text'] for text in trad_en[:]]).dropna()

      number = ((file_trad_en.split('/')[-1]).split('.')[0]).split('_')[-1]

      df_trad.to_csv(self.path_out + str(number) + '.csv', index=False, encoding='utf-8')

### Traduzione

In [ ]:
# PARAMETRI GLOBALI
root_folder = 'drive/MyDrive/Traduzione_Dantesca/Data_Translate/Opere/Dante/'

# DATI
data_folder_name_originale = 'Originale'
filename_originale = 'ORIG_FIORE_*.txt'

data_folder_name_traduzione = 'Traduzione'
filenamepath_traduzione_en = 'TRAD_1_FIORE_*.txt'

DATA_PATH_ORIGINAL = os.path.abspath(os.path.join(root_folder, data_folder_name_originale))
orig_filenamepath = os.path.abspath(os.path.join(DATA_PATH_ORIGINAL, filename_originale))

DATA_PATH_TRANSLATE = os.path.abspath(os.path.join(root_folder, data_folder_name_traduzione))
trad_filenamepath_en = os.path.abspath(os.path.join(DATA_PATH_TRANSLATE, filenamepath_traduzione_en))

data_folder_out = 'drive/MyDrive/Traduzione_Dantesca/Data_Translate/data/'
DATA_PATH_OUT = os.path.abspath(os.path.join(data_folder_out, 'fiore_'))

In [ ]:
translate_fiore = Translate(orig_filenamepath=orig_filenamepath,
                              trad_filenamepath_en=trad_filenamepath_en,
                              translator_en=translator_en,
                              path_out=DATA_PATH_OUT)

translate_fiore.translate()